In [8]:
import pandas as pd
import numpy as np
from chefboost.training import Training
from sklearn import model_selection
df = pd.read_csv("wine_dataset.csv")
#df.rename(columns={"type":"Decision"},inplace=True)
#config = {"algorithm":"ID3"}
#gains = Training.findGains(df,config)
#print(gains)
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]
for index,value in X.iterrows():
    print(value['alcohol'])

9.2
10.2
9.5
9.5
9.2
9.6
11.4
9.5
12.5
9.5
9.4
10.6
11.2
11.2
9.3
10.8
9.5
9.9
11.6
11.0
13.4
9.4
9.6
10.0
9.6
9.0
12.0
9.4
9.4
9.3
9.6
10.9
10.9
10.7
9.9
10.5
11.7
9.8
9.4
11.2
10.1
9.8
10.4
10.0
9.4
9.5
10.9
10.7
9.1
12.15
10.0
9.1
10.0
10.1
10.8
12.0
9.4
9.7
12.5
9.9
10.1
9.8
9.3
10.4
10.1
11.3
9.5
9.5
11.2
9.3
9.1
12.1
12.8
9.4
11.0
12.75
9.8
9.4
12.5
9.6
9.4
12.7
11.2
9.4
11.9
9.0
9.4
11.3
10.8
9.6
11.7
10.8
11.1
11.6
10.1
11.8
10.0
8.7
9.7
10.0
9.8
11.3
11.7
10.5
10.3
9.4
9.8
10.1
11.3
11.8
10.4
12.1
8.9
10.5
11.6
9.5
11.1
9.5
9.6
10.6
12.4
11.8
9.5
10.9
9.0
10.5
12.5
10.0
12.1
11.3
10.7
9.4
9.7
11.4
11.2
10.5
9.9
9.6
11.3
9.6
10.4
9.5
10.5
9.5
11.0
8.8
9.4
9.4
11.2
9.9
12.2
9.3
8.9
11.6
9.5
12.0
9.5
9.1
9.55
10.2
14.0
10.0
11.1
9.8
12.1
10.5
11.3
9.1
10.5
9.4
12.5
10.4
10.0
9.9
12.6
13.1
11.1
12.3
11.0
10.8
12.9
12.8
12.6
9.5
13.3
10.1
10.1
11.0
10.8
9.5
10.0
10.6
11.0
10.7
9.8
11.4
12.2
11.4
10.7
10.3
11.8
9.8
10.5333333333333
9.7
9.4
10.5
9.5
10.9
9.5
10.3
10.5
9.4
9.5
9.5
9.3

In [19]:
class Tree():
    def __init__(self):
        head = None
    def __str__(self):
        return f'Head: {self.head}'
    #classes for construction of tree
    class Node():
            def __init__(self,dec_key,dec_val):
                self.dec_key = dec_key
                self.dec_val = dec_val
            def __str__(self):
                return f'Decision key: {self.dec_key}, Desicion value: {self.dec_val}\n \tless than: {self.left_branch}\n \tgreater than: {self.right_branch}'
            def set_right(self,next):
                self.right_branch = next
            def set_left(self,next):
                self.left_branch = next
            def set_prev(self,prev):
                self.prev = prev
            pass
    class Leaf():
        def __init__(self,val):
            self.val = val
            pass
        def __str__(self):
            return f'Leaf value: {self.val}'
        def set_prev(self,prev):
            self.prev = prev
        pass

    def impurity(self,X,y,impurity_measure):
        impurity_list = []
        for key in X.keys():
            split = np.mean(X[key])
            X_split = X[key] < split
            impurity = 0
            for value in X_split.unique():
                prob_X = X_split.value_counts()[value]/len(X_split)
                if impurity_measure == 'gini':
                    impurity += prob_X
                column = y.keys()[-1]
                for label in y[column].unique():
                    try:
                        prob_y_cond = (((X_split == value)&(y[column] == label)).value_counts()[True]/len(X_split))/prob_X
                    except:
                        impurity_list.append(0)
                        continue
                    if impurity_measure == 'entropy':
                        impurity -= prob_X * prob_y_cond * np.log2(prob_y_cond)
                    elif impurity_measure == 'gini':
                        impurity -= prob_X * (prob_y_cond**2)
            impurity_list.append(impurity)
        impurity_list = np.array(impurity_list)
        return impurity_list

    def information_gain(self,X,y,impurity_measure):
        #calculate entropy of X data set - not actually necessary can just minimize conditional entropy
        #calculate conditional entropy given a split data for each predictor
        impurity_array = self.impurity(X,y,impurity_measure)
        if impurity_array.all() == 0:
            return self.Leaf(y[y.keys()[-1]].value_counts().keys()[0])
        #difference in data set entropy and conditional entropies gives us information gain of each split
        return np.argmin(impurity_array)

    def learn(self,X,y,impurity_measure = 'entropy',pruning = False):
        if pruning:
            X_training = X.iloc[:len(X)//2,:]
            X_pruning = X.iloc[len(X)//2:,:]
            y_training = y.iloc[:len(y)//2,:]
            y_pruning = y.iloc[len(y//2):,:]
            self.head = self.learn_recursive(X_training,y_training,impurity_measure)
        else:
            self.head = self.learn_recursive(X,y,impurity_measure)

    def learn_recursive(self,X,y,impurity_measure,prev = None):
        #check if all labels are the same; return leaf of that value if they are
        column = y.keys()[-1]
        if len(y[column].unique()) == 1:
            branch = self.Leaf(y[column].iloc[0])
            branch.set_prev(prev)
            return branch
        #check if all data values are the same and return most common label if true
        data_is_equal = True
        for key in X.keys():
            if len(X[key].unique()) == 1:
                continue
            else:
                data_is_equal = False
        if data_is_equal:
            branch = self.Leaf(y[column].value_counts().keys()[0])
            branch.set_prev(prev)
            return branch
        
        #find split with most information gain and split data to left and right branches
        else:
            split_index = self.information_gain(X,y,impurity_measure)
            if isinstance(split_index,self.Leaf):
                split_index.set_prev(prev)
                return split_index
            split_key = X.keys()[split_index]
            split_value = np.mean(X[split_key])
            branch = self.Node(split_key,split_value)
            branch.set_prev(prev)
            branch.set_left(self.learn_recursive(X[X[split_key] < split_value],y[X[split_key] < split_value],impurity_measure))
            branch.set_right(self.learn_recursive(X[X[split_key] >= split_value],y[X[split_key] >= split_value],impurity_measure))
            return branch

    def predict(self,x):
        results = []
        for index, value in x.iterrows():
            pointer = self.head
            while not isinstance(pointer,self.Leaf):
                dec_val = pointer.dec_val
                dec_key = pointer.dec_key
                if value[dec_key] < dec_val:
                    pointer = pointer.left_branch
                else:
                    pointer = pointer.right_branch
            results.append(pointer.val)
        return pd.DataFrame({"results":results})

In [21]:
df = pd.read_csv("wine_dataset.csv")
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]
#split data into training, validation, and test sets
#figure out how to set random seed
np.random.seed()
X_training,X_val_test,y_training,y_val_test = model_selection.train_test_split(X,y,train_size = 0.7,test_size=0.3)
X_validation,X_test,y_validation,y_test = model_selection.train_test_split(X_val_test,y_val_test,train_size = 0.5,test_size=0.5)
print(X_validation)
#training
model_0 = Tree()
model_0.learn(X_training,y_training)
y_predict = model_0.predict(X_validation)

      citric acid  residual sugar    pH  sulphates  alcohol
1214         0.31             3.9  3.15       0.31     11.5
1056         0.10             2.1  3.29       0.75     10.9
2706         0.48             1.8  3.30       1.05      9.4
1961         0.27             2.0  3.78       0.64     12.3
223          0.26             7.7  3.17       0.45      8.8
...           ...             ...   ...        ...      ...
1189         0.48             2.1  3.26       0.62     12.4
1437         0.49             2.5  3.36       0.78     11.0
688          0.48             1.8  3.39       0.53      9.4
1899         0.00             2.2  3.25       0.50      9.8
633          0.23             1.7  3.29       0.65      9.3

[480 rows x 5 columns]


In [24]:
(y_predict == y_validation).value_counts()[True]

,type
1214,True
1056,True
2706,True
1961,True
223,True
...,...
1189,True
1437,True
688,True
1899,True
